# Verify Properties of $J_{12}$

In this notebook, we verify some of the claimed properties of $J_{12}$. That is, we show:
- $J_{12}$ is not amenable;
- $J_{12}$ is simple spectrum;
- there exists doubly stochastic $X$ that commutes with the adjacency matrix of $J_{12}$; and,
- the same holds for a jellyfishification---specifically $J_3(J_{12})$.

In [4]:
from sympy import Matrix, MatrixSymbol, Poly, nsimplify, shape, BlockMatrix, ZeroMatrix
from sympy.solvers import solve
from scipy.optimize import linprog
import networkx as nx
import colour_refinement as CR

## Create the Adjacency matrix

In [ ]:
J_12 = nx.Graph()
J_12.add_nodes_from(range(1, 13))
J_12.add_edges_from([
    [1, 2],
    [1, 8],
    [2, 3], 
    [2, 8],
    [3, 4],
    [3, 9],
    [4, 5],
    [4, 12],
    [5, 6],
    [5, 11],
    [6, 7],
    [7, 8],
    [7, 10],
    [9, 10],
    [10, 11],
    [11, 12]
])
adj_J_12 = Matrix(nx.adjacency_matrix(J_12, nodelist=sorted(J_12.nodes)).todense())

## Show $J_{12}$ is not amenable

In [6]:
CR.is_amenable(J_12)

Fails A


False

## Show $J_{12}$ is simple spectrum

In [7]:
all(mult == 1 for eig, mult in adj_J_12.eigenvals().items())

True

## Find a doubly stochastic matrix that commutes with the adjacency matrix of $J_{12}$ 
This one requires a bit more work

### Define the constraints for doubly stochastic matrices

In [8]:
# 12 x 12 symbolic matrix
D = MatrixSymbol('x', 12, 12).as_explicit()

e = Matrix.ones(12,1)

For a matrix $D \in R^{n\times n}$ to be doubly stochastic, the sum of entries within each row and within each column must be equal to 1. Give the matrix $e = [1, 1, \dots, 1]^T \in R^{n \times 1}$, these constraints can be expressed as

$$
De=D^Te=e.
$$

Furthermore, we wish to find a doubly stochastic matrix that commutes with our adjacency matrix $M$. This constraint can be expressed as

$$
MD-DM=\bf{0}.
$$

In [9]:
doubly_stochastic_soln = solve([(D*e-e),D.transpose()*e-e, adj_J_12*D-D*adj_J_12],D, positive=True)

In [10]:
# The matrix obtained by replacing dependent variables with the equations that define their dependencies
D_dep = D.xreplace(doubly_stochastic_soln)
D_dep

Matrix([
[-11*x[11, 2]/23 - 10*x[11, 3]/23 + 10*x[11, 4]/23 + 32*x[11, 5]/23 - 8*x[11, 6]/23 + 14*x[11, 7]/23 + 43*x[11, 8]/23 - 17*x[11, 9]/23 + 6*x[11, 10]/23 + 48*x[11, 11]/23 - 25/23,      34*x[11, 2]/23 + 33*x[11, 3]/23 + 36*x[11, 4]/23 - 9*x[11, 5]/23 + 31*x[11, 6]/23 + 32*x[11, 7]/23 + 3*x[11, 8]/23 + 17*x[11, 9]/23 + 40*x[11, 10]/23 - 2*x[11, 11]/23 + 2/23,            -15*x[11, 2]/23 + x[11, 3]/23 - x[11, 4]/23 + 6*x[11, 5]/23 - 13*x[11, 6]/23 - 6*x[11, 7]/23 + 21*x[11, 8]/23 + 4*x[11, 9]/23 - 19*x[11, 10]/23 + 9*x[11, 11]/23 - 9/23,        2*x[11, 2]/23 - 17*x[11, 3]/23 - 6*x[11, 4]/23 + 13*x[11, 5]/23 - 9*x[11, 6]/23 + 10*x[11, 7]/23 + 11*x[11, 8]/23 - 22*x[11, 9]/23 + x[11, 10]/23 + 8*x[11, 11]/23 - 8/23, -3*x[11, 2]/23 - 9*x[11, 3]/23 - 14*x[11, 4]/23 - 8*x[11, 5]/23 + 2*x[11, 6]/23 - 15*x[11, 7]/23 - 28*x[11, 8]/23 - 13*x[11, 9]/23 - 13*x[11, 10]/23 - 12*x[11, 11]/23 + 12/23,        4*x[11, 2]/23 + 12*x[11, 3]/23 + 11*x[11, 4]/23 + 3*x[11, 5]/23 + 5*x[11, 6]/23 + 20*x[11, 

### Find an explicit solution
With the dependencies defined, we now wish to find an explicit solution. The first step is find the _free_ variables, i.e. the variables with no dependencies.

In [11]:
free = sorted(list(set(D)-set(doubly_stochastic_soln.keys())), key = lambda a: int(str(a).split()[1][:-1]))
free

[x[11, 2],
 x[11, 3],
 x[11, 4],
 x[11, 5],
 x[11, 6],
 x[11, 7],
 x[11, 8],
 x[11, 9],
 x[11, 10],
 x[11, 11]]

Then we create the matrix of coefficients for the linear system that ensures each entry in $D$ is between $0$ and $1$. Specifically, we observe that $a_1x_1 + a_2x_2 + \dots + a_mx_m + k\leq 1$ is equivalent to $a_1x_1 + a_2x_2 + \dots + a_mx_m \leq 1-k$, and $a_1x_1 + a_2x_2 + \dots + a_mx_m + k\geq 0$ is equivalent to $-a_1x_1 - a_2x_2 - \dots - a_mx_m\leq k$.

In [12]:
pos_coeffs = Matrix([[Poly(d, free).coeff_monomial(var) for var in free] for d in D_dep])
neg_coeffs = -1*pos_coeffs
upper_bound = Matrix([[1-Poly(d,free).coeff_monomial(1)] for d in D_dep])
lower_bound = Matrix([[Poly(d,free).coeff_monomial(1)] for d in D_dep])

constraint_coeffs = Matrix.vstack(pos_coeffs, neg_coeffs)
constraint_bounds = Matrix.vstack(upper_bound, lower_bound)

To find an explicit solution, we will use a linear programming solver. This solver requires a _target_ function; a function the solver tries to minimise over the feasible region our constraints have defined. For our purpose, we don't care what this function is, so we arbitrarily set it $0$.

In [13]:
c = [0 for d in free]

Now we have everything necessary to find a solution:

In [14]:
explicit_soln = linprog(c, constraint_coeffs, constraint_bounds, bounds=(0,1))

# Substitute values into matrix
subst = {f: nsimplify(explicit_soln.x[i]) for (i,f) in enumerate(free)}
D_explicit = D_dep.xreplace(subst)
D_explicit

Matrix([
[1/7,   0,   0,   0,   0, 2/7,   0,   0, 2/7,   0,   0, 2/7],
[  0,   0, 1/7, 1/7, 1/7,   0, 1/7, 1/7,   0, 1/7, 1/7,   0],
[  0, 1/7,   0, 1/7, 1/7,   0, 1/7, 1/7,   0, 1/7, 1/7,   0],
[  0, 1/7, 1/7,   0, 1/7,   0, 1/7, 1/7,   0, 1/7, 1/7,   0],
[  0, 1/7, 1/7, 1/7,   0,   0, 1/7, 1/7,   0, 1/7, 1/7,   0],
[2/7,   0,   0,   0,   0, 1/7,   0,   0, 2/7,   0,   0, 2/7],
[  0, 1/7, 1/7, 1/7, 1/7,   0,   0, 1/7,   0, 1/7, 1/7,   0],
[  0, 1/7, 1/7, 1/7, 1/7,   0, 1/7,   0,   0, 1/7, 1/7,   0],
[2/7,   0,   0,   0,   0, 2/7,   0,   0, 1/7,   0,   0, 2/7],
[  0, 1/7, 1/7, 1/7, 1/7,   0, 1/7, 1/7,   0,   0, 1/7,   0],
[  0, 1/7, 1/7, 1/7, 1/7,   0, 1/7, 1/7,   0, 1/7,   0,   0],
[2/7,   0,   0,   0,   0, 2/7,   0,   0, 2/7,   0,   0, 1/7]])

As a check, we show that $MD=DM$

In [15]:
adj_J_12*D_explicit==D_explicit*adj_J_12

True

## Create $J_3(J_{12})$

In [ ]:
J_12_3 = CR.jellyfishify(J_12, 3)
adj_J_12_3 = Matrix(nx.adjacency_matrix(J_12_3, nodelist = sorted(J_12_3.nodes)).todense())

## Show $J_3(J_{12})$ is not amenable

In [17]:
CR.is_amenable(J_12_3)

Fails A


False

## Show $J_3(J_{12})$ is simple spectrum

In [18]:
all(mult == 1 for eig, mult in adj_J_12_3.eigenvals().items())

True

## Find a doubly stochastic matrix that commutes with the adjacency matrix of $J_{12}$ 
This time, rather than going through the arduous process of searching for such a matrix, we instead use the construction presented in the proof of Lemma 4.3.7

In [19]:
Z = Matrix(ZeroMatrix(12, 12))
X = D_explicit

# Construct X_3 as in the proof of Lemma 4.3.7
X_3 = Matrix(BlockMatrix([
    [X, Z, Z, Z],
    [Z, X, Z, Z],
    [Z, Z, X, Z],
    [Z, Z, Z, X],
]))

Check that it's doubly stochastic:

In [23]:
e_48 = Matrix.ones(48, 1)
X_3*e_48 == X_3.transpose()*e_48 == e_48

True

Check that it commutes with the adjacency matrix:

In [24]:
X_3*adj_J_12_3 == adj_J_12_3*X_3

True